In [1]:
# Problem 1 - Build the BWT

def makeBwt(t):
    """Create the BWT for the string t$"""
    tDollar = t + "$"
    
    #generate all of the cyclic rotations of the string and sort them lexographically 
    #then take the last character of each, and join that list
    thisBWT = str("".join([string[-1] for string in sorted([tDollar[start:] + tDollar[0:start] for start in range(0, len(tDollar))])]))
    
    return thisBWT

text = "GATTACA"

bwt = makeBwt(text)

bwt == "ACTGA$TA"

True

In [2]:
# Problem 2 - Invert the BWT

def invertBwt(bwt):
    """Inverts the Burrows-Wheeler Transform, returning the original string using 
    inefficent algorithm"""
  
  #Iteration (number of times = length of the BWT) of the following... 
        #given array of the k-mers up the known column 
        #append the BWT to each
        #sort that array
    
    #initializing cyclic rotation array with a list version of the bwt
    listBWT = list(bwt)
    kmerArray = list(bwt)
    
    for iteration in range(len(bwt)-1):
        #sorting the current sorted array
        currentColumn = sorted(kmerArray)
        
        #pairwise appending BWT to each element of the current column 
        newColumn = [listBWT[i]+currentColumn[i] for i in range(len(bwt))]
        #saving the new column as the next kmerArray
        kmerArray = newColumn
    
    #is this the right way to do this????
    inverseBWT = sorted(kmerArray)[0]
    hit = inverseBWT[1:] + inverseBWT[0]
    
    
    
    
    return hit
text = "GATTACA"
bwt = "ACTGA$TA"

invertBwt(bwt) == text + "$"

True

In [9]:
# Problem 3 - Complete Last-to-First mapping using FM-index


#complete
class FmIndex(object):
    def __init__(self, t, alphabet):
        ''' Create FM-index for t in naive manner '''
      
        # Make the BWT 
        # We don't compress or anything here, but you could
        self.bwt = makeBwt(t)
        self.L = self.bwt
      
        # Calculate C lookup dictionary in crappy way
        #Nathaniel's notes: 
        #sorts the BWT to return column L.
        #Since the BWT is sorted, iterating backwards, each character's index is its C[i] lookup value 
        s = sorted(self.bwt)
        
        self.C = {}
        for i in range(len(s)-1, -1, -1):
            self.C[s[i]] = i
        #print(self.C)
      
      # Calculate full Occ table in similarly crappy way
      # Recall, this is not using sampling and is therefore very memory inefficient 
    #Nathaniel's notes:
    #Builds a list of dictionaries
    #Iterates, for a given index in the BWT, over all the characters and counts occurences
        self.Occ = [ {} for i in range(len(self.bwt)) ]
        for i in range(len(self.bwt)):
            for j in alphabet + "$":
                #below line initializes the dictionary with occurence values of 0 for 0th index
                p = self.Occ[i-1][j] if i > 0 else 0
                #uses the occurence value of the character at the last index to calculate the current one 
                self.Occ[i][j] = p + (1 if self.bwt[i] == j else 0)
        #print(self.Occ)
      
    def lf(self, i):
        """ Return the last-to-first mapping for index i of the bwt """
    #Code to complete
    #Hint, don't forget to convert between 0 based Python string coordinates and 1 based coordinates of FM-index tables
    
    #uses the formula LF[i] = C[L[i]] + Occ[L[i], i]
    
        LF_i = int(self.C[self.L[i]] + self.Occ[i][self.L[i]] - 1)
        
        
        
        return LF_i
    
dnaAlphabet = "ACGT"
fmIndex = FmIndex(text, dnaAlphabet)

print(fmIndex)


fmIndex.lf(5) == 0

True

In [17]:
# Problem 4 - Use backward search to invert the BWT

def invertBwtUsingFmIndex(fmIndex):
    """ Returns t by using repeated lf search to reconstruct t$ backwards"""
  
  # Code to write
  ## Hint: start from location of "$"  in bwt, then walk backwards using fmIndex.lf
  ## function to build t
    
    #starting w/location of $
    originalString = "$"
    lastIndex = fmIndex.bwt.index("$")
    
    #iterating through all but the last character of the bwt
    for preceedingCharacterIndex in range(len(fmIndex.bwt)-1):
        #using last to first mapping using the lf backwards walk
        #finds the index, in the BWT, of the preceeding character
        indexPreceedingCharacter = fmIndex.lf(lastIndex)
        #saving that index as the last index, to start the walk from again
        lastIndex = indexPreceedingCharacter
        
        #going to that character in the bwt, and adding it the the front of the substring
        originalString = fmIndex.bwt[indexPreceedingCharacter] + originalString
    
    return originalString
        
        

invertBwtUsingFmIndex(fmIndex) == "GATTACA$"

True

Problem 4 questions:

Q: What is the asymptotic runtime of your invertBwtUsingFmIndex function?
m = length of BWT

O(m) time. 

Q2: How does this to compare to the runtime of the invertBWT function?

Invert BWT function uses two nested for loops which means O(m^2) time, a whole factor of m higher. 

In [ ]:
# Extra credit: adapt the FmIndex class and add a pattern query method to search for instance of input pattern in t